In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import csv

def is_static_file(url):
    static_extensions = [
        '.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg', '.webp', '.ico',
        '.mp4', '.avi', '.mov', '.wmv', '.flv', '.mkv',
        '.mp3', '.wav', '.ogg', '.aac',
        '.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx', '.txt',
        '.zip', '.rar', '.tar', '.gz',
        '.css', '.js'
    ]
    try:
        parsed_url = urlparse(url)
        path = parsed_url.path.lower()
        if not path or path == '/':
            return False
        return any(path.endswith(ext) for ext in static_extensions)
    except Exception:
        return True

def is_valid_domain(url, domain):
    try:
        return urlparse(url).netloc == domain
    except Exception:
        return False

def crawl_data(start_url, domain, max_pages=None, output_csv="crawled_edges.csv"):
    visit_urls = [start_url]
    visited_urls = set()
    edge_list = set()

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    pages_crawled_count = 0

    while visit_urls:
        if max_pages is not None and pages_crawled_count >= max_pages:
            print(f"Reached limit of {max_pages} pages. Stopping crawl.")
            break
        
        url = visit_urls.pop(0)

        if url in visited_urls:
            continue

        parsed_url = urlparse(url)
        normalized_url = parsed_url.scheme + "://" + parsed_url.netloc + parsed_url.path.rstrip('/')
        if not parsed_url.path:
            normalized_url = parsed_url.scheme + "://" + parsed_url.netloc
        if parsed_url.query:
            normalized_url += "?" + parsed_url.query
        
        if normalized_url in visited_urls:
            continue

        print(f"Crawling ({pages_crawled_count + 1}): {normalized_url}")
        visited_urls.add(normalized_url)
        pages_crawled_count += 1

        try:
            time.sleep(0.3)
            response = requests.get(normalized_url, headers=headers, timeout=10)
            response.raise_for_status()

            content_type = response.headers.get('Content-Type', '').lower()
            if 'text/html' not in content_type:
                print(f"  Skipped (non-HTML): {normalized_url}")
                continue

            soup = BeautifulSoup(response.content, 'html.parser')

            for link_tag in soup.find_all('a', href=True):
                raw_href = link_tag['href']
                href = raw_href.split('#')[0]

                if not href or href.startswith(('mailto:', 'tel:', 'javascript:')):
                    continue

                absolute_url = urljoin(normalized_url, href)

                parsed_absolute_url = urlparse(absolute_url)
                normalized_dest_url = parsed_absolute_url.scheme + "://" + parsed_absolute_url.netloc + parsed_absolute_url.path.rstrip('/')
                if not parsed_absolute_url.path:
                    normalized_dest_url = parsed_absolute_url.scheme + "://" + parsed_absolute_url.netloc
                if parsed_absolute_url.query:
                    normalized_dest_url += "?" + parsed_absolute_url.query

                if is_valid_domain(normalized_dest_url, domain) and \
                   not is_static_file(normalized_dest_url):
                    
                    if normalized_dest_url not in visited_urls and \
                       normalized_dest_url not in visit_urls:
                        visit_urls.append(normalized_dest_url)
                    
                    if normalized_dest_url != normalized_url:
                        edge_list.add((normalized_url, normalized_dest_url))

        except requests.exceptions.RequestException as e:
            print(f"  Error crawling {normalized_url}: {e}")
        except Exception as e:
            print(f"  Unknown error processing {normalized_url}: {e}")

    print(f"\nCrawl completed. Total: {len(visited_urls)}")
    print(f"Total unique links (edges) found: {len(edge_list)}")

    if edge_list:
        print(f"\nSaving edge list to {output_csv}...")
        valid_edge_list = [(src, dst) for src, dst in edge_list if src and dst and isinstance(src, str) and isinstance(dst, str)]
        print(f"Valid edges after filtering: {len(valid_edge_list)}")
        
        with open(output_csv, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(["source_page", "destination_page"])
            writer.writerows(valid_edge_list)
        
        print(f"Saved edge list to {output_csv}")
    else:
        print("No links collected. No CSV file created.")

    return list(edge_list)

if __name__ == "__main__":
    start_url = "https://it.tdtu.edu.vn"
    domain = urlparse(start_url).netloc
    
    print(f"Crawling from: {start_url} (Domain: {domain})")
    crawled_edge_list = crawl_data(start_url, domain, max_pages=None)

Crawling from: https://it.tdtu.edu.vn (Domain: it.tdtu.edu.vn)
Crawling (1): https://it.tdtu.edu.vn
Crawling (2): https://it.tdtu.edu.vn/en
Crawling (3): https://it.tdtu.edu.vn/giao-vien
Crawling (4): https://it.tdtu.edu.vn/gioi-thieu
Crawling (5): https://it.tdtu.edu.vn/giao-duc
Crawling (6): https://it.tdtu.edu.vn/khoa-hoc-cong-nghe
Crawling (7): https://it.tdtu.edu.vn/tin-tuc-khoa
Crawling (8): https://it.tdtu.edu.vn/doanh-nghiep
Crawling (9): https://it.tdtu.edu.vn/tuyen-sinh
Crawling (10): https://it.tdtu.edu.vn/vien-chuc
  Error crawling https://it.tdtu.edu.vn/vien-chuc: 404 Client Error: Not Found for url: https://it.tdtu.edu.vn/vien-chuc
Crawling (11): https://it.tdtu.edu.vn/sinh-vien
  Error crawling https://it.tdtu.edu.vn/sinh-vien: 404 Client Error: Not Found for url: https://it.tdtu.edu.vn/sinh-vien
Crawling (12): https://it.tdtu.edu.vn/iccies
Crawling (13): https://it.tdtu.edu.vn/tuyen-sinh/thac-si-nganh-khoa-hoc-may-tinh
Crawling (14): https://it.tdtu.edu.vn/truong-dai-ho

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

class PageRank:
    def __init__(self, max_iterations=20, damping_factor=0.85, epsilon=1e-6):
        self.max_iterations = max_iterations
        self.damping_factor = damping_factor
        self.epsilon = epsilon
        self.ranks = None

    def compute_pagerank(self, edges_df):
        links = edges_df.select(
            F.col("source_page").alias("src"),
            F.col("destination_page").alias("dst")
        ).distinct()

        nodes = links.select("src").union(links.select("dst")) \
                     .distinct().withColumnRenamed("src", "url")

        num_nodes = nodes.count()
        num_edges = links.count()
        if num_nodes == 0:
            print("No nodes to compute PageRank.")
            self.ranks = spark.createDataFrame([], nodes.schema.add("pagerank", "double"))
            return self.ranks

        print(f"Total number of unique pages (nodes): {num_nodes}")
        print(f"Total number of edges: {num_edges}")

        ranks = nodes.withColumn("pagerank", F.lit(1.0 / num_nodes))
        out_degrees_df = links.groupBy("src").agg(F.count("dst").alias("out_degree"))
        d = self.damping_factor

        for i in range(self.max_iterations):
            print(f"Iteration {i + 1}/{self.max_iterations}")

            contributions = links.join(ranks, links.src == ranks.url, "inner") \
                                .join(out_degrees_df, links.src == out_degrees_df.src, "inner") \
                                .select(links.dst.alias("url"),
                                        (ranks.pagerank / out_degrees_df.out_degree).alias("contribution"))

            aggregated_contrib = contributions.groupBy("url").agg(F.sum("contribution").alias("total_contribution"))

            new_pageranks = nodes.join(aggregated_contrib, "url", "left_outer") \
                                 .select(F.col("url"),
                                         ((1.0 - d) / num_nodes + d * F.coalesce(F.col("total_contribution"), F.lit(0.0))).alias("new_pagerank"))
            
            if i % 5 == 0 or i == self.max_iterations - 1:
                diff = new_pageranks.join(ranks, "url", "inner") \
                                    .select(F.sum(F.abs(new_pageranks.new_pagerank - ranks.pagerank)).alias("l1_diff")) \
                                    .first()["l1_diff"]
                print(f"L1 diff: {diff}")
                if diff is not None and diff < self.epsilon:
                    print(f"Converged after {i + 1} iterations (L1 diff: {diff})")
                    break
            else:
                diff = None

            ranks = new_pageranks.select(F.col("url"), F.col("new_pagerank").alias("pagerank"))


        rank_sum = ranks.agg(F.sum("pagerank").alias("total")).collect()[0]["total"]
        if rank_sum > 0:
            ranks = ranks.withColumn("pagerank", F.col("pagerank") / rank_sum)

        self.ranks = ranks.orderBy(F.col("pagerank").desc())
        return self.ranks

if __name__ == "__main__":
    spark = SparkSession.builder \
        .appName("PageRank") \
        .master("local[*]") \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.memory", "4g") \
        .getOrCreate()

    input_file = "crawled_edges.csv"
    print(f"Reading edges from {input_file}...")

    try:
        edges_spark_df = spark.read.csv(input_file, header=True, inferSchema=True) \
                      .select("source_page", "destination_page") \
                      .filter((F.col("source_page").isNotNull()) & (F.col("destination_page").isNotNull())) \
                      .cache()
        if edges_spark_df.count() == 0:
            print("No edges found in the CSV file.")
            spark.stop()
            exit()

        print("Sample Data:")
        edges_spark_df.show(5, truncate=False)

        print("\nComputing PageRank...")
        pagerank = PageRank(max_iterations=20, damping_factor=0.85, epsilon=1e-6)
        page_ranks_df = pagerank.compute_pagerank(edges_spark_df)

        if page_ranks_df and page_ranks_df.count() > 0:
            print("\nPageRank Results:")
            page_ranks_df.show(10, truncate=False)

            most_important_page = page_ranks_df.first()
            if most_important_page:
                print(f"The most important page is: {most_important_page['url']} with PageRank: {most_important_page['pagerank']:.6f}")
        else:
            print("No PageRank results found.")

    except Exception as e:
        print(f"Error reading or processing {input_file}: {e}")

    spark.stop()

Reading edges from crawled_edges.csv...
Sample Data:
+-----------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------+
|source_page                                                                                                                                    |destination_page                             |
+-----------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------+
|https://it.tdtu.edu.vn/user/login?destination=/sinh-vien-khoa-cntt-tham-gia-lop-hoc-agile-scrum-cung-doanh-nghiep%23comment-form               |https://it.tdtu.edu.vn/giao-duc              |
|http://it.tdtu.edu.vn/user/login?destination=/ekino-vietnam-vi-tri-manual-tester-fresher-junior-co-hoi-nghe-nghiep-cho-sinh-vien%23comment-form|http://it.tdtu.edu.vn/khoa-hoc-con